# Skip notebook test
-----

#### NOTE:  This notebook will take hours to run.
-----



# Comparing NetworkX vs cuGraph using synthetic data on various algorithms


This notebook compares the execution times of many of the cuGraph and NetworkX algorithms when run against identical synthetic data at multiple scales.

This notebook uses the RMAT data generator which allows the creation of graphs at various scales.  The notebook, by default, runs on a set of selected sizes but users are free to change or add to that list.

Notebook Credits

    
| Author        |    Date    |  Update             | cuGraph Version |  Test Hardware         |
| --------------|------------|---------------------|-----------------|------------------------|
| Don Acosta    | 1/12/2023  | Created             | 23.02 nightly   | RTX A6000, CUDA 11.7   |
| Brad Rees     | 1/27/2023  | Modified            | 23.02 nightly   | RTX A6000, CUDA 11.7   |


### Timing 

When looking at the overall workflow, NetworkX and cuGraph do things differently.  For example, NetworkX spends a lot of time creating the graph data structure.  cuGraph on the other hand does a lazy creation of the data structure when an algorithm is called.  To further complicate the comparison problem, NetworkX does not always return the answer.  In some cases, it returns a generator that is then called to produce the data.  

This benchmark produces two performance metrics:
 - (1)	Just the algorithm run time 
 - (2)	The algorithm plus graph creation time

Since GPU memory is a precious resource, having a lot of temporary data laying around is avoided.  So once a graph is created, the raw data is dropped.  
 
__What is not timed__:  Generating the data with R-MAT</p>
__What is timed__:     (1) creating a Graph, (2) running the algorithm (3) run any generators


### Algorithms

|        Algorithm        |  Type         | Undirected Graph | Directed Graph |   Notes
| ------------------------|---------------|------ | ------- |-------------
| Katz                    | Centrality    |   X   |         | 
| Betweenness Centrality  | Centrality    |   X   |         | Estimated, k = 100
| Louvain                 | Community     |   X   |         | Uses python-louvain for comparison
| Triangle Counting       | Community     |   X   |         |
| Core Number             | Core          |   X   |         |
| PageRank                | Link Analysis |       |    X    |
| Jaccard                 | Similarity    |   X   |         |
| BFS                     | Traversal     |   X   |         | No depth limit
| SSSP                    | Traversal     |   X   |         | 


### Test Data
Data is generated using a Recursive MATrix (R-MAT) graph generation algorithm. 
The generator specifics are documented [here](https://docs.rapids.ai/api/cugraph/stable/api_docs/generator.html)



### Notes
* Running Betweenness Centrality on the full graph is prohibitive using NetworkX.  Anything over k=100 can explode runtime to days


## Import Modules

In [1]:
# system and other
import gc
import os
from time import perf_counter
import numpy as np
import math

# rapids
import cugraph
import cudf

# to parallelize with dask
import dask_cudf
from cugraph.dask.common.mg_utils import get_visible_devices
from cugraph.testing.mg_utils import start_dask_client, stop_dask_client

# NetworkX libraries
import networkx as nx

# RMAT data generator
from cugraph.generators import rmat
from cugraph.structure import NumberMap

In [2]:
try: 
    import community
except ModuleNotFoundError:
    os.system('pip install python-louvain')
    import community

### Determine the scale of the test data
RMAT generates graph where the number of vertices is a power of 2 and the number of edges is based on an edge factor times the number vertices.

Since RMAT tends to generate about 50% isolated vertices, those vertices are dropped from the graph data.  Hence the number of vertices is closer to (2 ** scale) / 2


| Scale | Vertices (est) | Edges  |
| ------|----------------|--------|
| 10 | 512 | 16,384 | 
| 11 | 1,024 | 32,768| 
| 12 | 2,048 | 65,536| 
| 13 | 4,096 | 131,072| 
| 14 | 8,192 | 262,144| 
| 15 | 16,384 | 524,288 | 
| 16 | 32,768 | 1,048,576 | 
| 17 | 65,536 | 2,097,152 | 
| 18 | 131,072 | 4,194,304 | 
| 19 | 262,144 | 8,388,608 | 
| 20 | 524,288 | 16,777,216 | 
| 21 | 1,048,576 | 33,554,432 | 
| 22 | 2,097,152 | 67,108,864 | 
| 23 | 4,194,304 | 134,217,728 | 
| 24 | 8,388,608 | 268,435,456 | 
| 25 | 16,777,216 | 536,870,912 | 


In [3]:
# Test Data Sizes
# Here you can create an array of test data sizes.   Then set the "data" variable to the array you want
# the dictionary format is 'name' : scale


# These scales are used by R-MAT to determine the number of vertices/edges in the synthetic data graph.
data_full = {
    'data_scale_10'   :  10,
    'data_scale_12'   :  12,
    'data_scale_14'  :   14,
    'data_scale_16'  :   16,
    'data_scale_18'  :   18,
    'data_scale_20'  :   20,
}

# for quick testing
data_quick = {
   'data_scale_9' : 9,
   'data_scale_10' : 10,
   'data_scale_11' : 11,
}


# Which dataset is to be used
data = data_quick


### Generate data
The data is generated once for each size.

In [4]:
# Data generator 
#  The result is an edgelist of the size determined by the scale and edge factor
def generate_data(scale, edgefactor=16, mg=False):
    _gdf = rmat(
        scale,
        (2 ** scale) * edgefactor,
        0.57,
        0.19,
        0.19,
        42,
        clip_and_flip=False,
        scramble_vertex_ids=True,
        create_using=None,  # return edgelist instead of Graph instance
        mg=mg # determines whether generated data will be used on one or multiple GPUs
        )

    clean_coo = NumberMap.renumber(_gdf, src_col_names="src", dst_col_names="dst")[0]
    if mg:
        clean_coo.rename(columns={"renumbered_src": "src", "renumbered_dst": "dst"})
    else:
        clean_coo.rename(columns={"renumbered_src": "src", "renumbered_dst": "dst"}, inplace=True)

    print(f'Generated a dataframe of type {type(clean_coo)}, with {len(clean_coo)} edges')
    
    return clean_coo

## Create Graph functions
There are two types of graphs created:
* Directed Graphs - calls to create_nx_digraph, create_cu_directed_graph.
* Undirected Graphs - calls to create_xx_ugraph <- fully symmeterized

In [5]:
# NetworkX
def create_nx_graph(_df, directed=False):
    t1 = perf_counter()
    if directed:
        g_type = nx.DiGraph
    else:
        g_type = nx.Graph
        
    _gnx = nx.from_pandas_edgelist(_df,
                            source='src',
                            target='dst',
                            edge_attr=None,
                            create_using=g_type)
    t2 = perf_counter() - t1

    return _gnx, t2

# cuGraph
def create_cu_graph(_df,transpose=False, directed=False, mg=False):
    t1 = perf_counter()
    _g = cugraph.Graph(directed=directed)

    if mg:
        # Set the number of partition to #GPUs
        npartitions = len(get_visible_devices())
        _ddf = dask_cudf.from_cudf(_df.compute(), npartitions=npartitions)
        _g.from_dask_cudf_edgelist(_ddf, source="src", destination="dst", edge_attr=None)
    else:
        _g.from_cudf_edgelist(_df,
                            source='src',
                            destination='dst',
                            edge_attr=None,
                            renumber=False,
                            store_transposed=transpose)
    t2 = perf_counter() - t1

    return _g, t2

## Algorithm Execution

### Katz

In [6]:
def nx_katz(_G, alpha):
    t1 = perf_counter()
    _ = nx.katz_centrality(_G, alpha)
    t2 = perf_counter() - t1
    return t2

def cu_katz(_G, alpha, mg=False):
    t1 = perf_counter()
    if mg:
        _ = cugraph.dask.katz_centrality(_G, alpha)
    else:

        _ = cugraph.katz_centrality(_G, alpha)
    t2 = perf_counter() - t1
    return t2


### Betweenness Centrality

In [7]:
def nx_bc(_G, _k):
    t1 = perf_counter()
    _ = nx.betweenness_centrality(_G, k=_k)
    t2 = perf_counter() - t1
    return t2

def cu_bc(_G, _k, mg=False):
    t1 = perf_counter()
    if mg:
        _ = cugraph.dask.betweenness_centrality(_G, k=_k)
    else:   
        _ = cugraph.betweenness_centrality(_G, k=_k)
    t2 = perf_counter() - t1
    return t2


### Louvain

In [8]:
def nx_louvain(_G):
    t1 = perf_counter()
    parts = community.best_partition(_G)
    
    # Calculating modularity scores for comparison
    _ = community.modularity(parts, _G)
    
    t2 = perf_counter() - t1
    return t2

def cu_louvain(_G, mg=False):
    t1 = perf_counter()
    if mg:
        _, modularity = cugraph.dask.louvain(_G)
        print (f'modularity: {modularity}')
    else:
        _,_ = cugraph.louvain(_G)
    t2 = perf_counter() - t1
    return t2



### Triangle Counting

In [9]:
def nx_tc(_G):
    t1 = perf_counter()
    nx_count = nx.triangles(_G)

    # To get the number of triangles, we would need to loop through the array and add up each count
    count = 0
    for key, value in nx_count.items():
        count = count + value
    
    t2 = perf_counter() - t1
    return t2

def cu_tc(_G, mg=False):
    t1 = perf_counter()
    if mg:
        _ = cugraph.dask.triangle_count(_G)
    else:
        _ = cugraph.triangle_count(_G)
    t2 = perf_counter() - t1
    return t2


### Core Number

In [10]:
def nx_core_num(_G):
    t1 = perf_counter()
    _G.remove_edges_from(nx.selfloop_edges(_G))
    nx_count = nx.core_number(_G)
    
    count = 0
    for key, value in nx_count.items():
        count = count + value
    
    t2 = perf_counter() - t1
    return t2

def cu_core_num(_G, mg=False):
    t1 = perf_counter()
    if mg:
        _ = cugraph.dask.core_number(_G)
    else:
        _ = cugraph.core_number(_G)
    t2 = perf_counter() - t1
    return t2


### PageRank

In [11]:
def nx_pagerank(_G):
    t1 = perf_counter()
    _ = nx.pagerank(_G)
    t2 = perf_counter() - t1
    return t2 

def cu_pagerank(_G, mg=False):
    t1 = perf_counter()
    if mg:
        _ = cugraph.dask.pagerank(_G)
    else:
        _ = cugraph.pagerank(_G)
    t2 = perf_counter() - t1
    return t2


### Jaccard

In [12]:
def nx_jaccard(_G):
    t1 = perf_counter()
    nj = nx.jaccard_coefficient(_G)
    t2 = perf_counter() - t1
    return t2

def cu_jaccard(_G, mg=False):
    t1 = perf_counter()
    t1 = perf_counter()
    if mg:
        _ = cugraph.dask.jaccard(_G)
    else:
        _ = cugraph.jaccard_coefficient(_G)
    t2 = perf_counter() - t1
    return t2


### BFS

In [13]:
def nx_bfs(_G):
    seed = 0
    t1 = perf_counter()
    nb = nx.bfs_edges(_G, seed)
    nb_list = list(nb) # gen -> list
    t2 = perf_counter() - t1
    return t2

def cu_bfs(_G, mg=False):
    seed = 0
    t1 = perf_counter()
    if mg:
        _ = cugraph.dask.bfs(_G, seed)
    else:
        _ = cugraph.bfs(_G, seed)
    t2 = perf_counter() - t1
    return t2


### SSSP

In [14]:
def nx_sssp(_G):
    seed = 0
    t1 = perf_counter()
    _ = nx.shortest_path(_G, seed)
    t2 = perf_counter() - t1
    return t2

def cu_sssp(_G, mg=False):
    seed = 0
    t1 = perf_counter()
    # SSSP requires weighted graph
    if mg:
        _ = cugraph.dask.bfs(_G, seed)
    else:
        _ = cugraph.bfs(_G, seed)
    t2 = perf_counter() - t1
    return t2


---

# Benchmark

In [15]:
# number of datasets
num_datasets = len(data)

In [16]:
# arrays to capture performance gains
names = []
algos = []
graph_create_cu = []
graph_create_nx = []

# Two dimension data [file, perf]
time_algo_nx = []          # NetworkX
time_algo_cu = []          # cuGraph
perf = []
perf_algo = []

algos.append("   ")

i = 0
for k,v in data.items():
    # init all the 2-d arrays
    time_algo_nx.append([])
    time_algo_cu.append([])
    perf.append([])
    perf_algo.append([])

    # Saved the file Name
    names.append(k)

    # generate data
    print("------------------------------")
    print(f'Creating Graph of Scale = {v}')

    gdf = generate_data(v)
    pdf = gdf.to_pandas()
    print(f"\tdata in gdf {len(gdf)} and data in pandas {len(pdf)}")

    # create the graphs
    g_cu, tcu = create_cu_graph(gdf)
    g_nx, tnx = create_nx_graph(pdf)
    graph_create_cu.append(tcu)
    graph_create_nx.append(tnx)
    del gdf, pdf

    # prep
    deg = g_cu.degree()
    deg_max = deg['degree'].max()

    alpha = 1 / deg_max
    num_nodes = g_cu.number_of_vertices()

    del deg
    gc.collect()

    #----- Algorithm order is same as defined at top ----

    #-- Katz 
    print("\tKatz  ", end = '')
    if i == 0: 
        algos.append("Katz")

    print("n.", end='')
    tx = nx_katz(g_nx, alpha)
    print("c.", end='')
    tc = cu_katz(g_cu, alpha)
    print("")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf_algo[i].append ( (tx/tc) )
    perf[i].append( (tx + tnx) /  (tc + tcu) )

    #-- BC
    print("\tBC k=100  ", end='')
    if i == 0:
        algos.append("BC Estimate fixed")

    k = 100
    if k > num_nodes:
        k = int(num_nodes)
    print("n.", end='')
    tx = nx_bc(g_nx, k)
    print("c.", end='')
    tc = cu_bc(g_cu, k)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf_algo[i].append ( (tx/tc) )
    perf[i].append( (tx + tnx) /  (tc + tcu) )

    #-- Louvain
    print("\tLouvain  ", end='')
    if i == 0:
        algos.append("Louvain")

    print("n.", end='')
    tx = nx_louvain(g_nx)
    print("c.", end='')
    tc = cu_louvain(g_cu)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf_algo[i].append ( (tx/tc) )
    perf[i].append( (tx + tnx) /  (tc + tcu) )

    #-- TC
    print("\tTC  ", end='')
    if i == 0:
        algos.append("TC")

    print("n.", end='')
    tx = nx_tc(g_nx)
    print("c.", end='')
    tc = cu_tc(g_cu)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf_algo[i].append ( (tx/tc) )
    perf[i].append( (tx + tnx) /  (tc + tcu) )

    #-- Core Number
    print("\tCore Number  ", end='')
    if i == 0:
        algos.append("Core Number")

    print("n.", end='')
    tx = nx_core_num(g_nx)
    print("c.", end='')
    tc = cu_core_num(g_cu)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf_algo[i].append ( (tx/tc) )
    perf[i].append( (tx + tnx) /  (tc + tcu) )

    #-- PageRank
    print("\tPageRank  ", end='')
    if i == 0:
        algos.append("PageRank")

    print("n.", end='')
    tx = nx_pagerank(g_nx)
    print("c.", end='')
    tc = cu_pagerank(g_cu)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf_algo[i].append ( (tx/tc) )
    perf[i].append( (tx + tnx) /  (tc + tcu) )

    #-- Jaccard
    print("\tJaccard  ", end='')
    if i == 0:
        algos.append("Jaccard")

    print("n.", end='')
    tx = nx_jaccard(g_nx)
    print("c.", end='')
    tc = cu_jaccard(g_cu)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf_algo[i].append ( (tx/tc) )
    perf[i].append( (tx + tnx) /  (tc + tcu) )

    #-- BFS
    print("\tBFS  ", end='')
    if i == 0:
        algos.append("BFS")

    print("n.", end='')
    tx = nx_bfs(g_nx)
    print("c.", end='')
    tc = cu_bfs(g_cu)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf_algo[i].append ( (tx/tc) )
    perf[i].append( (tx + tnx) /  (tc + tcu) )

    #-- SSSP
    print("\tSSSP  ", end='')
    if i == 0:
        algos.append("SSP")

    print("n.", end='')
    tx = nx_sssp(g_nx)
    print("c.", end='')
    tc = cu_sssp(g_cu)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf_algo[i].append ( (tx/tc) )
    perf[i].append( (tx + tnx) /  (tc + tcu) )

    # increament count
    i = i + 1
    
    del g_cu, g_nx
    gc.collect()


------------------------------
Creating Graph of Scale = 9
Generated a dataframe of type <class 'cudf.core.dataframe.DataFrame'>, with 8192 edges
	data in gdf 8192 and data in pandas 8192
	Katz  n.c.
	BC k=100  n.c.

/home/mnaim/miniconda3/envs/cuda_12_new_2/lib/python3.10/site-packages/cugraph/centrality/katz_centrality.py:121: UserWarning: Katz centrality expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


 
	Louvain  n.c. 
	TC  n.c. 
	Core Number  n.c. 
	PageRank  n.c. 
	Jaccard  n.c. 
	BFS  n.c. 
	SSSP  n.c. 
------------------------------
Creating Graph of Scale = 10
Generated a dataframe of type <class 'cudf.core.dataframe.DataFrame'>, with 16384 edges
	data in gdf 16384 and data in pandas 16384


/home/mnaim/miniconda3/envs/cuda_12_new_2/lib/python3.10/site-packages/cugraph/link_analysis/pagerank.py:227: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


	Katz  n.c.
	BC k=100  n.

...

/home/mnaim/miniconda3/envs/cuda_12_new_2/lib/python3.10/site-packages/cugraph/link_analysis/pagerank.py:227: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


c.
	BC k=100  n.

/home/mnaim/miniconda3/envs/cuda_12_new_2/lib/python3.10/site-packages/cugraph/centrality/katz_centrality.py:121: UserWarning: Katz centrality expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


c. 
	Louvain  n.c. 
	TC  n.c. 
	Core Number  n.c. 
	PageRank  n.c. 
	Jaccard  n.c. 
	BFS  n.c. 
	SSSP  n.c. 


/home/mnaim/miniconda3/envs/cuda_12_new_2/lib/python3.10/site-packages/cugraph/link_analysis/pagerank.py:227: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


In [17]:
#Print results
print(algos)

for i in range(num_datasets):
    print(f"{names[i]}")
    print(f"{perf[i]}")
    print(f"{perf_algo[i]}")

['   ', 'Katz', 'BC Estimate fixed', 'Louvain', 'TC', 'Core Number', 'PageRank', 'Jaccard', 'BFS', 'SSP']
data_scale_9
[0.30174025119666154, 0.5252496066735675, 0.4815963183980694, 0.10059243531734213, 0.06274260384596582, 0.08087691640849742, 0.04259226789052634, 0.04541648848958121, 0.04630903277395244]
[10.57018736844198, 1.0236825289491465, 3.4565544669058315, 1.7519735735549575, 0.20360754388242536, 1.996866060869272, 0.017142060522749525, 0.07449551793212239, 0.10215939753145187]
data_scale_10
[10.422613622706065, 2.2596945327623845, 5.400742833397456, 2.670122216953594, 0.682892670528767, 2.511963848371226, 0.5618378459026585, 1.0027274366915082, 1.0377674050792682]
[27.48959075860895, 2.275587863157422, 6.2873877384903345, 3.4689708464108686, 0.3821397055328311, 3.8512864713871493, 0.0005884210429041734, 0.1674301749913076, 0.19472820374933392]
data_scale_11
[18.597918818212044, 4.662328129832614, 9.282539915015656, 5.826916762148887, 1.2560074010281037, 4.831561094416702, 0.68

In [18]:
#Print results
print("\n------------------------------")
print("\tAlgorithm Run times  (NX then cuGraph)\n")

print(algos)
for i in range(num_datasets):
    print(f"{names[i]}")
    print(f"{time_algo_nx[i]}")
    print(f"{time_algo_cu[i]}")


------------------------------
	Algorithm Run times  (NX then cuGraph)

['   ', 'Katz', 'BC Estimate fixed', 'Louvain', 'TC', 'Core Number', 'PageRank', 'Jaccard', 'BFS', 'SSP']
data_scale_9
[0.03168975654989481, 0.11808181460946798, 0.0607406310737133, 0.007159244269132614, 0.003216283395886421, 0.004577603191137314, 0.00012310687452554703, 0.00039636343717575073, 0.0004781009629368782]
[0.0029980316758155823, 0.11535003408789635, 0.017572594806551933, 0.004086388275027275, 0.015796484425663948, 0.002292393706738949, 0.007181568071246147, 0.005320634692907333, 0.004679950885474682]
data_scale_10
[0.07129270676523447, 0.26657472364604473, 0.1288990117609501, 0.01734251156449318, 0.008041061460971832, 0.009121773764491081, 7.294118404388428e-06, 0.0009355107322335243, 0.0010360050946474075]
[0.0025934437289834023, 0.11714543215930462, 0.020501202903687954, 0.004999324679374695, 0.021042203530669212, 0.002368500456213951, 0.012396086938679218, 0.005587467923760414, 0.005320262163877487]

## Example MG runs

In [22]:

# Setting up cluter
setup_objs = start_dask_client()

gdf = generate_data(10, mg=True)
# create graph
g_cu, tcu = create_cu_graph(gdf, mg=True)
del gdf

# prep
deg = g_cu.degree()
deg_max = deg['degree'].max().compute()

alpha = 1 / deg_max
num_nodes = g_cu.number_of_vertices()
k = 100 if num_nodes > 100 else num_nodes

tc = cu_katz(g_cu, alpha, mg=True)
tc = cu_bc(g_cu, k, mg=True)
tc = cu_louvain(g_cu, mg=True)
tc = cu_tc(g_cu, mg=True)
tc = cu_core_num(g_cu, mg=True)
tc = cu_pagerank(g_cu, mg=True)
tc = cu_jaccard(g_cu, mg=True)
tc = cu_bfs(g_cu, mg=True)



# Tearing down the cluster
stop_dask_client(*setup_objs)

/home/mnaim/miniconda3/envs/cuda_12_new_2/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37567 instead
  warnings.warn(



Dask client/cluster created using LocalCUDACluster


Exception: Communicator is already initialized

___
Copyright (c) 2020-2023, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___